## RNN 이해

### RNN의 약자?
* 순환 신경망(Recurrent Neural Network)이다.


### 어떤 경우에 쓰일까?
* 앞의 정보로 다음에 나오는 정보 추측. RNN을쓰면 좋은 프로그램을 만들 수 있다.

### 대표적인 서비스
* 2016년 구글의 신경망 기반 기계 번역이 RNN을 이용해서 만들었다.

### 기존의 신경망과 다른 점이 있다면

In [0]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./mnist/data/', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [0]:
### 옵션 설정
# learning_rate = 0.001
learning_rate = 0.01
total_epoch = 10
batch_size = 256

n_input = 28  # 입력
n_step = 28   # 28단계( 28 X 28 )
n_hidden = 256 # 은닉층 노드 수 
n_class = 10   # 클래스 10

## 실습
## 학습률 : 0.1, 0.01, 0.001
## total_epoch = 30, 50, 100
## batch_size = 64, 128, 256
## 은닉층 노드 수 : 128, 256

### Tensorflow 에서 함수 제공하여 RNN 셀 생성
* RNN 기본 신경망 다양한 셀을 사용할 수 있도록 함수를 제공
* RNN 단점 : 각 단계의 데이터를 학습할 때, 맨뒤에서는 맨 앞의 정보를 잘 기억할 수 없다.
* LSTM(Long Short-Term Memory)의 신경망
* GRU(Gated Recurrent Units) 약간 간단한 신경망

In [0]:
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.float32, [None, n_class])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

In [0]:
# BasicRNNCell, BasicLSTMCell, GRUCell
# tf.keras.layers.SimpleRNNCell 대체 예정 
cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
cell

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.


In [0]:
## 원래 
## states = tf.zeros(batch_size)
## for i in range(n_step):
##     output, states = cell(X[[:, i]], states)
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

In [0]:
# 결과를 Y의 다음 형식과 바꿔야 하기 때문에
# Y : [batch_size, n_class]
# outputs 의 형태를 이에 맞춰 변경해야합니다.
# outputs : [batch_size, n_step, n_hidden]
# -> [n_step, batch_size, n_hidden]
# tf.transpose 함수를 이용하여 n_step과 batch_size의 차원의 순서를 바꾸고,
# n_step 차원을 제거하여 마지막 단계의 결과값을 취함.
outputs = tf.transpose(outputs, [1, 0, 2])
# -> [batch_size, n_hidden]
outputs = outputs[-1]
# y = x * W + b를 이용하여 최종 결과값.
model = tf.matmul(outputs, W) + b


In [0]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [0]:
#########
# 신경망 모델 학습
######
sess = tf.Session()
sess.run(tf.global_variables_initializer())
total_batch = int(mnist.train.num_examples/batch_size)

In [0]:
%%time

# total_epoch : 30
# batch_size : 128
# n_step : 28
# n_input : 28
for epoch in range(total_epoch):
  total_cost = 0
  for i in range(total_batch):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    # X 데이터를 RNN 입력 데이터에 맞게 [batch_size, n_step, n_input] 형태로 변환합니다.
    batch_xs = batch_xs.reshape((batch_size, n_step, n_input))
    _, cost_val = sess.run([optimizer, cost],
    feed_dict={X: batch_xs, Y: batch_ys})
    total_cost += cost_val
  print('Epoch:', '%04d' % (epoch + 1),
        'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))
print('최적화 완료!')


Epoch: 0001 Avg. cost = 3.906
Epoch: 0002 Avg. cost = 2.360
Epoch: 0003 Avg. cost = 2.356
Epoch: 0004 Avg. cost = 2.295
Epoch: 0005 Avg. cost = 2.162
Epoch: 0006 Avg. cost = 2.366
Epoch: 0007 Avg. cost = 2.364
Epoch: 0008 Avg. cost = 2.375
Epoch: 0009 Avg. cost = 2.368
Epoch: 0010 Avg. cost = 2.355
최적화 완료!
CPU times: user 6min 2s, sys: 14.9 s, total: 6min 17s
Wall time: 3min 21s


In [0]:
#########
# 결과 확인
######
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print(is_correct)
print(accuracy)


Tensor("Equal:0", shape=(?,), dtype=bool)
Tensor("Mean_1:0", shape=(), dtype=float32)


In [0]:
# test_batch_size : 10000, n_step : 28, n_input : 28
test_batch_size = len(mnist.test.images)
test_xs = mnist.test.images.reshape(test_batch_size, n_step, n_input)
test_ys = mnist.test.labels

In [0]:
print('정확도:', sess.run(accuracy,
 feed_dict={X: test_xs, Y: test_ys}))

정확도: 0.0959
